In [1]:
import rvnewop as rv
from glob import glob
import matplotlib.pyplot as plt
from itertools import accumulate

In [2]:
# remove st.hst, edn.hst, minver.hst, cubic.hst, wikisort.hst, nbody.hst
floating_point_hsts = list(map(lambda x: "./embench_hst/"+x, ["st.hst", "edn.hst", "minver.hst", "cubic.hst", "wikisort.hst", "nbody.hst"]))
for FILENAME in [x for x in glob("./embench_hst/*.hst") if x not in floating_point_hsts]:
    #print("Processing {}...".format(FILENAME))

    prog = rv.Histogram.parse(FILENAME, isa="32ICM")
    prog.findBasicBlocks()

    prog.addLivenessValuesToGraph()

    # total cycles = sum of all the frequencies
    total_cycles = sum(
        [inst.freq for bb in prog.basicBlocks for inst in bb.bbInstructions()]
    )

    new_instructions = []

    for i, b in enumerate((prog.getSubBlocks())):
        graph = b.constructDAG()

        # temp will store every permutation of candidate subgraphs
        temp = []
        for n in rv.analysis.findCandidateSubgraphs(prog, graph):
            subtree = rv.analysis.createSubtreeFromNode(graph, n)
            root = n

            temp.append(rv.Subgraph(subtree, root))

        # add candidate subgraph with highest score to new instructions list
        current_nodes = set()
        temp = sorted(temp, key=lambda sg: sg.score, reverse=True)
        while temp:
            if any([(n in current_nodes) for n in temp[0].graph.nodes]):
                temp.pop(0)
            else:
                current_nodes.update(temp[0].graph.nodes)
                new_instructions.append(temp.pop(0))

    new_instructions = sorted(new_instructions, key=lambda sg: sg.score, reverse=True)

    inst_dict = {}
    for subgraph in new_instructions:
        key = rv.analysis.graphToParenString(subgraph.graph)

        if key in inst_dict:
            inst_dict[key].score += subgraph.score
        else:
            inst_dict[key] = subgraph

    new_instructions = [inst_dict[key] for key in inst_dict]
    new_instructions = sorted(new_instructions, key=lambda sg: sg.score, reverse=True)

    #max_depth = 0
    #max_mult_depth = 0
    depth_hst = {x:0 for x in range(8)}
    for sg in new_instructions:
        
        depth_hst[min(sg.depth, 7)] += sg.score
        
        #if sg.containsMultiplyInstruction():
        #    max_mult_depth = max(max_mult_depth, sg.depth)
        #else:
        #    max_depth = max(max_depth, sg.depth)

    saved_cycles = sum([sg.score for sg in new_instructions])
    percent_cycles = float(saved_cycles) / total_cycles * 100
    #plt.clf()
    #plt.plot([x for x in range(len(new_instructions))],list(map(lambda x:x/total_cycles, accumulate([sg.score for sg in new_instructions]))))
    #plt.show()
    #print(
    #    """Saved {} cycles out of {} ({:.4}%) with {} new instructions""".format(
    #        saved_cycles, total_cycles, percent_cycles, len(new_instructions)
    #    )
    #)
    #print("max depth: {}\nmax mult depth: {}".format(max_depth, max_mult_depth))
    
    print("{},{:.4},{:.4},{:.4},{:.4},{:.4},{:.4},{:.4},{:.4}".format(FILENAME[len("./embench_hst/"):-4], *[depth_hst[key]/saved_cycles for key in range(8)]))


aha-mont64,0.0,0.03118,0.6975,0.0668,0.2007,0.003726,0.0001226,0.0
ud,0.0,2.065e-06,0.053,0.6708,0.2759,0.0002092,6.883e-06,0.0
slre,0.0,0.0,0.09977,0.3403,0.5561,0.003706,0.0001235,0.0
qrduino,0.0,0.0,0.2864,0.01877,0.2472,0.341,0.1066,5.374e-05
matmult-int,0.0,4.959e-05,1.282e-05,0.9969,0.002184,4.187e-05,0.0008373,0.0
picojpeg,0.0,0.0,0.2419,0.3274,0.02261,0.1132,0.0001649,0.2947
sglib-combined,0.0,0.02065,0.6026,0.162,0.02768,0.1272,0.0,0.05991
nsichneu,0.0,0.0,0.006456,0.1471,0.8308,0.01513,0.0005044,0.0
coremark,0.0,0.0,0.1094,0.04001,0.8465,0.003461,2.394e-05,0.0005937
nettle-sha256,0.0,0.0,0.3084,0.6775,0.01391,0.0002043,6.72e-06,0.0
huffbench,0.0,2.784e-05,0.3646,0.6276,0.007636,0.0001522,0.0,0.0
crc32,0.0,0.0,3.325e-05,0.001097,0.5356,0.4633,3.779e-06,0.0
nettle-aes,0.0,0.0,0.02477,0.9382,0.02418,0.01285,8.14e-06,0.0
statemate,0.0,0.0001524,0.004469,0.1476,0.8318,0.01544,0.0005079,0.0
